# Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
import optuna

rng = np.random.RandomState(42)

# Data

In [3]:
data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features.csv'
data = pd.read_csv(data_path)
data['DATE'] = data['DATE'].astype('datetime64[ns]')

X_train = data.loc[data['DATE'].dt.year <= 2021].drop(columns=['TARGET', 'DATE']).copy()
y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
X_valid = data.loc[data['DATE'].dt.year == 2022].drop(columns=['TARGET', 'DATE']).copy()
y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
X_test = data.loc[data['DATE'].dt.year == 2023].drop(columns=['TARGET', 'DATE']).copy()
y_test = data.loc[data['DATE'].dt.year == 2023, 'TARGET'].copy()

print(f'Train: {X_train.shape}, {y_train.shape}')
print(f'Valid: {X_valid.shape}, {y_valid.shape}')
print(f'Test: {X_test.shape}, {y_test.shape}')

Train: (34938, 52), (34938,)
Valid: (2920, 52), (2920,)
Test: (2920, 52), (2920,)


# Baseline

In [3]:
model = CatBoostRegressor(verbose=100, random_state=42)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Learning rate set to 0.071783
0:	learn: 10.1222909	total: 60.3ms	remaining: 1m
100:	learn: 3.9090917	total: 604ms	remaining: 5.38s
200:	learn: 3.8082690	total: 1.14s	remaining: 4.54s
300:	learn: 3.7281364	total: 1.74s	remaining: 4.05s
400:	learn: 3.6601188	total: 2.28s	remaining: 3.4s
500:	learn: 3.6011532	total: 2.79s	remaining: 2.78s
600:	learn: 3.5470066	total: 3.3s	remaining: 2.19s
700:	learn: 3.5004606	total: 3.84s	remaining: 1.64s
800:	learn: 3.4523707	total: 4.38s	remaining: 1.09s
900:	learn: 3.4076202	total: 4.92s	remaining: 541ms
999:	learn: 3.3676583	total: 5.43s	remaining: 0us
Train MAE: 2.5486400384277177
Valid MAE: 3.1525589942969856


# Hyperparameter tuning

In [4]:
def load_data():
    data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features.csv'
    data = pd.read_csv(data_path)
    data['DATE'] = data['DATE'].astype('datetime64[ns]')
    X_train = data.loc[data['DATE'].dt.year <= 2021].drop(columns=['TARGET', 'DATE']).copy()
    y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
    X_valid = data.loc[data['DATE'].dt.year == 2022].drop(columns=['TARGET', 'DATE']).copy()
    y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
    return X_train, X_valid, y_train, y_valid

def objective(trial):
    PARAMS = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-6, 0.5),
        'depth': trial.suggest_int('depth', 1, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0, 100.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 100.0), # CPU only
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 100.0),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise']),
    }
    X_train, _, y_train, _ = load_data()
    model = CatBoostRegressor(verbose=0, random_state=42)
    model.set_params(**PARAMS)
    score = np.mean(cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_absolute_error'))
    return score

In [5]:
sampler = optuna.samplers.TPESampler(
    n_startup_trials = 50, # the random sampling is used instead of the TPE algorithm until the given number of trials finish in the same study
    n_ei_candidates = 24, # number of candidate samples used to calculate the expected improvement.
    multivariate = True, # multivariate TPE when suggesting candidates; default: False
    seed = 42
)
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=100, interval_steps=20),
    direction='maximize',
    sampler=sampler,
    storage='sqlite:///db.sqlite3'
)
study.optimize(
    objective, n_trials = 1000,
    timeout = 3600*10, # in seconds
    n_jobs = 1,
    show_progress_bar = True
)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-12-19 18:58:49,149] A new study created in RDB with name: no-name-cdcb4056-f60e-45fe-bde0-54ea64c37568


  0%|          | 0/1000 [00:00<?, ?it/s]

[I 2024-12-19 18:59:41,079] Trial 0 finished with value: -3.250370032781568 and parameters: {'n_estimators': 406, 'learning_rate': 0.47535720249065166, 'depth': 9, 'l2_leaf_reg': 59.86584841970366, 'random_strength': 15.601864044243651, 'bagging_temperature': 15.599452033620265, 'grow_policy': 'Depthwise'}. Best is trial 0 with value: -3.250370032781568.
[I 2024-12-19 18:59:45,641] Trial 1 finished with value: -3.0618915233279203 and parameters: {'n_estimators': 621, 'learning_rate': 0.3540365808254449, 'depth': 1, 'l2_leaf_reg': 96.99098521619943, 'random_strength': 83.24426408004217, 'bagging_temperature': 21.233911067827616, 'grow_policy': 'Depthwise'}. Best is trial 1 with value: -3.0618915233279203.
[I 2024-12-19 18:59:57,467] Trial 2 finished with value: -3.1169885715849035 and parameters: {'n_estimators': 339, 'learning_rate': 0.26237869105968725, 'depth': 6, 'l2_leaf_reg': 29.122914019804192, 'random_strength': 61.18528947223795, 'bagging_temperature': 13.949386065204184, 'grow

In [5]:
model = CatBoostRegressor(
    n_estimators=759, learning_rate=0.042, depth=5,
    l2_leaf_reg=27.37, random_strength=0.081, bagging_temperature=99.92, 
    grow_policy='SymmetricTree', verbose=100, random_state=42
)
model.fit(X_train, y_train)
print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

0:	learn: 10.3730001	total: 59.1ms	remaining: 44.8s
100:	learn: 3.9480427	total: 629ms	remaining: 4.1s
200:	learn: 3.8960275	total: 1.12s	remaining: 3.11s
300:	learn: 3.8647117	total: 1.63s	remaining: 2.48s
400:	learn: 3.8364129	total: 2.15s	remaining: 1.92s
500:	learn: 3.8119391	total: 2.66s	remaining: 1.37s
600:	learn: 3.7908350	total: 3.17s	remaining: 833ms
700:	learn: 3.7735699	total: 3.68s	remaining: 305ms
758:	learn: 3.7624864	total: 3.98s	remaining: 0us
Train MAE: 2.813072372535432
Valid MAE: 3.1266897640579234


In [6]:
model = CatBoostRegressor(
    n_estimators=750, depth=5,
    l2_leaf_reg=27, random_strength=0.08, bagging_temperature=100, 
    grow_policy='SymmetricTree', verbose=100, random_state=42
)
model.fit(X_train, y_train)
print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

0:	learn: 10.4745073	total: 6.63ms	remaining: 4.97s
100:	learn: 4.0296333	total: 577ms	remaining: 3.71s
200:	learn: 3.9157674	total: 1.09s	remaining: 2.97s
300:	learn: 3.8900955	total: 1.58s	remaining: 2.35s
400:	learn: 3.8695494	total: 2.09s	remaining: 1.82s
500:	learn: 3.8509830	total: 2.61s	remaining: 1.29s
600:	learn: 3.8329821	total: 3.16s	remaining: 785ms
700:	learn: 3.8176988	total: 3.67s	remaining: 257ms
749:	learn: 3.8097525	total: 3.92s	remaining: 0us
Train MAE: 2.8451949618561176
Valid MAE: 3.1359811318965263
